In [1]:
import sys
sys.executable
sys.path

['/home/sakthy1497/Documents/Comment_Analysis/Sentiment-Analysis-for-Comments',
 '/home/sakthy1497/anaconda3/envs/intent_classification/lib/python37.zip',
 '/home/sakthy1497/anaconda3/envs/intent_classification/lib/python3.7',
 '/home/sakthy1497/anaconda3/envs/intent_classification/lib/python3.7/lib-dynload',
 '',
 '/home/sakthy1497/anaconda3/envs/intent_classification/lib/python3.7/site-packages',
 '/home/sakthy1497/anaconda3/envs/intent_classification/lib/python3.7/site-packages/IPython/extensions',
 '/home/sakthy1497/.ipython']

In [2]:
#import import_ipynb
import nltk
nltk.download()
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [ ]:
np.random.seed(500)

In [ ]:
Corpus = pd.read_csv(r"/home/sakthy1497/Downloads/Text-Classification-master/corpus.csv",encoding='latin-1')

In [11]:
# Step - a : Remove blank rows if any.
Corpus['text'].dropna(inplace=True)

# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['text'] = [entry.lower() for entry in Corpus['text']]

# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['text']= [word_tokenize(entry) for entry in Corpus['text']]

# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

for index,entry in enumerate(Corpus['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/home/sakthy1497/snap/jupyter/6/nltk_data'
    - '/snap/jupyter/6/nltk_data'
    - '/snap/jupyter/6/share/nltk_data'
    - '/snap/jupyter/6/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [23]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['label'],test_size=0.3)

In [24]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [26]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [27]:
print(Tfidf_vect.vocabulary_)

{'stun': 4271, 'even': 1529, 'sound': 4116, 'track': 4556, 'beautiful': 387, 'paint': 3160, 'mind': 2839, 'well': 4865, 'would': 4951, 'recomend': 3598, 'people': 3225, 'hate': 2054, 'video': 4763, 'game': 1855, 'music': 2925, 'play': 3298, 'cross': 1032, 'ever': 1533, 'best': 427, 'back': 330, 'away': 320, 'crude': 1035, 'take': 4371, 'fresh': 1812, 'step': 4210, 'guitar': 2004, 'soulful': 4113, 'orchestra': 3101, 'impress': 2245, 'anyone': 203, 'care': 645, 'listen': 2611, 'soundtrack': 4118, 'anything': 204, 'read': 3564, 'lot': 2655, 'review': 3721, 'say': 3840, 'figure': 1694, 'write': 4957, 'disagree': 1238, 'bit': 449, 'ultimate': 4633, 'masterpiece': 2756, 'timeless': 4506, 'year': 4976, 'beauty': 389, 'simply': 4011, 'refuse': 3621, 'price': 3407, 'tag': 4370, 'pretty': 3402, 'must': 2931, 'go': 1927, 'buy': 604, 'cd': 676, 'much': 2916, 'money': 2878, 'one': 3079, 'feel': 1671, 'worth': 4947, 'every': 1535, 'penny': 3224, 'amaze': 152, 'favorite': 1660, 'time': 4505, 'hand': 

In [29]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)
print(predictions_NB)

Naive Bayes Accuracy Score ->  83.23333333333333
[0 0 1 ... 1 1 0]
